In [ ]:
import os
import requests
import pandas as pd

def dlmergnhanes(arrsurfix, arryears, arrData, master):
    """
    Download and merge datasets from several years of the NHANES and return a dictionary with the merged data.

    Parameters:
        arrsurfix (list): Array of surfix corresponding to each year.
        arryears (list): Array of cycles to download.
        arrData (list): Array of datasets to download from each year cycle.
        master (int): Master flag.

    Returns:
        dict: Merged data.

    Example:
        arrsurfix = ['D', 'E', 'F', 'G', 'H']
        arryears = ['2005-2006', '2007-2008', '2009-2010', '2011-2012', '2013-2014']
        arrData = ['DEMO', 'BPX', 'BMX']
        master = 1

        # Set the data directory
        data_dir = 'C:\\Users\\Desktop\\dl_merge_data_NHANES\\data'

        # Call the function to download and merge data
        DM = dlmergnhanes(arrsurfix, arryears, arrData, master)
    """

    totalPat = 0  # total subjects
    DM = {}

    for k in range(len(arryears)):
        nhsYrs = arryears[k]
        surfix = arrsurfix[k]

        # Create directory if not exists
        data_dir = 'C:\\Users\\simcich\\Desktop\\dl_merge_data_NHANES\\data'
        os.makedirs(data_dir, exist_ok=True)

        # Download data from CDC
        for i in range(len(arrData)):
            wData = arrData[i]
            url = f'https://wwwn.cdc.gov/Nchs/Nhanes/{nhsYrs}/{wData}_{surfix}.XPT'
            filename = os.path.join(data_dir, f'{wData}_{surfix}.XPT')
            response = requests.get(url)
            with open(filename, 'wb') as f:
                f.write(response.content)

        # Merge data
        wData = arrData[0]
        filename = os.path.join(data_dir, f'{wData}_{surfix}.XPT')
        tempData = pd.read_sas(filename)
        totalPat += tempData.shape[0]

        Data = {'years': nhsYrs}
        for i in range(len(arrData)):
            wData = arrData[i]
            filename = os.path.join(data_dir, f'{wData}_{surfix}.XPT')
            Data[wData] = pd.read_sas(filename)

        DM['test'] = '1'
        DM[f'merge{nhsYrs[:4]}'] = Data[arrData[0]]

        for i in range(len(arrData)):
            DM[f'merge{nhsYrs[:4]}'] = pd.merge(
                DM[f'merge{nhsYrs[:4]}'], Data[arrData[i]], how='left', left_on=DM[f'merge{nhsYrs[:4]}'].columns.tolist(),
                right_on=Data[arrData[i]].columns.tolist())

    return DM
